In [15]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 패키지 import

In [16]:
# 필요한 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
warnings.filterwarnings('ignore')

In [17]:
!pip install faiss-cpu
import faiss

# 1-1. Import Data

In [19]:
data = pd.read_csv('/content/drive/MyDrive/ChecKHUMate/merge_domitory_data.csv')

# 1-2. Data preprocessing

In [20]:
data

,user_id,domitory,age,student_id,gender,major,bedtime,clean_duration,smoke,alcohol,...,wish_student_id,wish_gender,wish_major,wish_bedtime,wish_clean_duration,wish_smoke,wish_alcohol,wish_mbti,wish_alarm,wish_activity
0,1,0,0.0,0,1,2,2,0,0.0,0,...,0,1,2,2,0,0.0,0,ENTJ,2.0,1.0
1,2,0,3.0,3,1,2,2,1,0.0,1,...,3,1,2,2,1,0.0,1,ISFP,1.0,2.0
2,3,0,3.0,3,1,2,2,0,1.0,1,...,3,1,2,2,0,1.0,1,ESTJ,2.0,2.0
3,4,0,0.0,0,1,2,2,1,0.0,0,...,0,1,2,2,1,0.0,0,ISFJ,2.0,1.0
4,5,0,3.0,3,0,0,2,0,0.0,0,...,3,0,0,2,0,0.0,0,ISFJ,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,129,0,4.0,4,1,3,3,1,0.0,0,...,4,1,3,3,1,0.0,0,ISFP,1.0,NaN
129,130,4,3.0,3,0,1,3,0,0.0,1,...,3,0,1,3,0,0.0,1,ISFJ,2.0,NaN
130,131,0,2.0,3,1,2,3,2,0.0,0,...,3,1,2,3,2,0.0,0,INFP,0.0,NaN
131,132,0,3.0,3,1,2,3,0,0.0,1,...,3,1,2,3,0,0.0,1,INTJ,1.0,NaN


In [21]:
data = data.set_index('user_id')

In [22]:
data.columns

Index(['domitory', 'age', 'student_id', 'gender', 'major', 'bedtime',
       'clean_duration', 'smoke', 'alcohol', 'mbti', 'alarm', 'activity',
       'wish_domitory', 'wish_age', 'wish_student_id', 'wish_gender',
       'wish_major', 'wish_bedtime', 'wish_clean_duration', 'wish_smoke',
       'wish_alcohol', 'wish_mbti', 'wish_alarm', 'wish_activity'],
      dtype='object')

In [23]:
user_data = data[['domitory', 'age', 'student_id', 'gender', 'major', 'bedtime', 'clean_duration', 'smoke', 'alcohol', 'mbti', 'alarm', 'activity']]

In [24]:
wish_data = data[['wish_domitory', 'wish_age', 'wish_student_id', 'wish_gender', 'wish_major', 'wish_bedtime', 'wish_clean_duration', 'wish_smoke', 'wish_alcohol', 'wish_mbti', 'wish_alarm', 'wish_activity']]

In [33]:
# 범주형 데이터 to 수치형 벡터 (one-hot encoding)

user_data_one_hot = {
  'domitory' : [0, 1, 2, 3, 4],
  'age' : [0, 1, 2, 3, 4],
  'student_id' : [0, 1, 2, 3, 4],
  'gender' : [0, 1],
  'major' : [0, 1, 2, 3, 4, 5],
  'bedtime' : [0, 1, 2, 3, 4],
  'clean_duration' : [0, 1, 2],
  'smoke' : [0, 1],
  'mbti' : ['ISTJ', 'ISFJ', 'INFJ', 'INTJ', 'ISTP', 'ISFP', 'INFP', 'INTP', 'ESTP', 'ESFP', 'ENFP', 'ENTP', 'ESTJ', 'ESFJ', 'ESFJ', 'ENFJ', 'ENTJ']
}

encoder = OneHotEncoder()
user_encode = encoder.fit_transform(user_data).toarray()

In [34]:
wish_data_one_hot = {
  'wish_domitory' : [0, 1, 2, 3, 4],
  'wish_age' : [0, 1, 2, 3, 4],
  'wish_student_id' : [0, 1, 2, 3, 4],
  'wish_gender' : [0, 1],
  'wish_major' : [0, 1, 2, 3, 4, 5],
  'wish_bedtime' : [0, 1, 2, 3, 4],
  'wish_clean_duration' : [0, 1, 2],
  'wish_smoke' : [0, 1],
  'wish_mbti' : ['ISTJ', 'ISFJ', 'INFJ', 'INTJ', 'ISTP', 'ISFP', 'INFP', 'INTP', 'ESTP', 'ESFP', 'ENFP', 'ENTP', 'ESTJ', 'ESFJ', 'ESFJ', 'ENFJ', 'ENTJ']
}

encoder = OneHotEncoder()
wish_encode = encoder.fit_transform(wish_data).toarray()

In [30]:
wish_encode

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

# 2. Data Filtering & Modeling

## 2-1 Faiss 유사도

In [39]:
# 벡터의 차원
d = user_encode.shape[1]

# L2 거리를 사용하는 인덱스 생성
index = faiss.IndexFlatL2(d)
# 인덱스에 데이터 추가
index.add(user_encode.astype('float32'))
index.add(wish_encode.astype('float32'))

# 가장 가까운 이웃의 수 (임의 설정)
k = 100

# 쿼리 벡터에 가장 가까운 이웃 검색
distances, indices = index.search(encoded_features.astype('float32'), k)

In [40]:
print(indices[0])

[  0 133   3   9 136 142  57  72 190 205  15  19  20  51 148 152 153 184
   2   4   6  11  13  16  22  23  46  67 101 122 135 137 139 144 146 149
 155 156 179 200 234 255   1   5  10  17  21  29  30  32  45  50  54  62
  64  65  73  74  76  82  86 112 126 127 130 131 134 138 143 150 154 162
 163 165 178 183 187 195 197 198 206 207 209 215 219 245 259 260 263 264
   7   8  12  14  18  25  26  27  28  33]


In [41]:
print(distances[0])

[ 0.  0.  4.  4.  4.  4.  8.  8.  8.  8. 10. 10. 10. 10. 10. 10. 10. 10.
 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.
 12. 12. 12. 12. 12. 12. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14.
 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14.
 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.]


## 2-2 사용자 가중치 별 sort 함수

In [ ]:
import numpy as np

class FeatureSorter:
    def __init__(self, encoded_data):
        self.encoded_data = encoded_data
        self.num_features = encoded_data.shape[1]

    def sort_features_by_weight(self, weights):
        # 가중치 배열의 크기를 feature 개수에 맞게 조정
        weights = np.pad(weights, (0, self.num_features - len(weights)), mode='constant')
        # feature들의 가중치를 계산
        weighted_sum = np.dot(self.encoded_data, weights)
        # 가중치를 기반으로 feature들을 정렬한 인덱스를 반환
        sorted_indices = np.argsort(weighted_sum)[::-1]  # 내림차순으로 정렬
        return sorted_indices

In [ ]:
# 예시로 사용자가 설정한 가중치
user_weights = np.array([0.8, 0.5, 0.6, 0.3, 0.9, 0.7, 0.4, 0.2])

# FeatureSorter 클래스를 이용하여 feature를 정렬
feature_sorter = FeatureSorter(encoded_features)
sorted_indices = feature_sorter.sort_features_by_weight(user_weights)

# 정렬된 feature들의 인덱스 출력
print("Sorted feature indices:", sorted_indices)

Sorted feature indices: [  0   3  15  10   9  39  52  32  53  57  33  82 114  22  23  24  25  40
  29  14  30 124  71  62 101  54  26 126  46 112  51  65  63  80  85  69
  76  74  60  59  45  58  20 130 127  31 129  43  41  27  35  37  36  98
  38  21  49  34   1   2   4   5   6   7  47   8  11  12  13  28  17  42
  18  19  16 132  66  73 110 115 121  86  84 123 125  77 122  72  68  61
 131 128  67  70  64  81  48  44  99  75  93 107 111 108 109 118 113 116
 117 119 120 105 106  94 104  88 102 100  97  96  87  92 103  90  50  56
  89  55  95  83  79  78  91]


# 2-3 규칙 기반 필터링

특정 규칙을 정의해서 사용자가 선호할만한 룸메이트를 추천하는 방식의 함수를 짜봤습니다. (ex. 흡연자는 비흡연자와 매칭하지 않음)

여기서는 아래와 같은 기준으로 코드를 짜봤습니다.

- 성별이 같은 사람끼리 매칭
- 흡연 여부가 같은 사람끼리 매칭
- 청소 주기가 같은 사람끼리 매칭
- 숙면 시간이 비슷한 사람끼리 매칭 (차이가 1 이하)
- 음주 빈도가 비슷한 사람끼리 매칭 (차이가 1 이하)

In [ ]:
# 사용자를 입력받아 추천하는 함수
def recommend_roommate(user_id):
    try:
        user = data.loc[user_id]
        candidates = data.drop(user_id)  # 현재 사용자 제외

        # 규칙에 따른 필터링
        candidates = candidates[candidates['gender'] == user['gender']]
        candidates = candidates[candidates['smoke'] == user['smoke']]
        candidates = candidates[candidates['clean_duration'] == user['clean_duration']]
        candidates = candidates[(candidates['bedtime'] - user['bedtime']).abs() <= 1]
        candidates = candidates[(candidates['alcohol'] - user['alcohol']).abs() <= 1]

        return candidates.index
    except KeyError:
        print("No user with that ID.")
        return None

# 특정 사용자에 대한 룸메이트 추천 결과 출력
print(recommend_roommate(1))

Index([7, 10, 58, 62, 73, 84, 88, 98, 99, 108, 132], dtype='int64', name='user_id')


# 2-4 MLP

MLP 기반으로 사용자 간의 호환성 점수를 출력하는 MLP 모델 구현
- 0 ~ 100점 사이로 나올 수 있도록 한다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(6, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.MLP(x)

# 모델 인스턴스 생성 및 옵티마이저 설정
model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# 모델 훈련 함수
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.view(-1, 1))
        loss.backward()
        optimizer.step()

# 모델 테스트 함수
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target.view(-1, 1)).item()

    test_loss /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss}')

In [ ]:
epochs = 200

# 모델 훈련 및 테스트
for epoch in range(epochs):  # 50 에포크
    train(model, device, train_loader, optimizer, criterion)
    print(f'Epoch {epoch}')
    test(model, device, test_loader, criterion)

# 2-5 GNN

- 범주형 데이터 One-Hot Encoding
- 그래프 노드: 각 사용자
- 그래프 엣지: 사용자 간의 잠재적 유사성

=> 학습된 노드 임베딩을 사용하여 코사인 유사도 or 유클리디안 거리 계산 후에 가장 유사한 사용자를 룸메이트로 추천하는 방식.

In [ ]:
!pip install torch-geometric
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_add_pool
import torch.nn.functional as F

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.5 MB/s eta 0:00:00


In [ ]:
def create_edge_index(features, domitory_idx, gender_idx):
    edge_sources = []
    edge_targets = []

    num_users = features.shape[0]
    domitory_features = features[:, domitory_idx]
    gender_features = features[:, gender_idx]

    # 모든 사용자 쌍을 검토하여 조건에 맞는 경우 엣지를 추가
    for i in range(num_users):
        for j in range(i + 1, num_users):
            if domitory_features[i] == domitory_features[j] and gender_features[i] == gender_features[j]:
                edge_sources.append(i)
                edge_targets.append(j)

    return torch.tensor([edge_sources, edge_targets], dtype=torch.long)


def create_data(encoded_features, domitory_idx, gender_idx):
    # 사용자 특성 벡터
    x = torch.tensor(encoded_features, dtype=torch.float)

    # 엣지 인덱스 생성
    edge_index = create_edge_index(np.array(encoded_features), domitory_idx, gender_idx)

    # 데이터 객체 생성
    data = Data(x=x, edge_index=edge_index)
    return data

# 예시 사용자 데이터 (one-hot 인코딩된 형태로 가정)
encoded_features = [
    [1, 0, 0, 1],  # 사용자 1: 기숙사 0, 성별 1
    [1, 0, 0, 1],  # 사용자 2: 기숙사 0, 성별 1
    [0, 1, 0, 0],  # 사용자 3: 기숙사 1, 성별 0
    [1, 0, 0, 0],  # 사용자 4: 기숙사 0, 성별 0
    [0, 1, 1, 0],  # 사용자 5: 기숙사 1, 성별 1
    [1, 0, 0, 0],  # 사용자 6: 기숙사 0, 성별 0
]

# 기숙사와 성별 인덱스 (one-hot 벡터 내 위치)
domitory_idx = [0, 1]  # 기숙사 정보가 첫 번째와 두 번째 위치에 있다고 가정
gender_idx = [2, 3]    # 성별 정보가 세 번째와 네 번째 위치에 있다고 가정

# 데이터 생성
data = create_data(encoded_features, domitory_idx, gender_idx)
print(data)


In [ ]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, num_features, hidden_dim):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

지금 우리가 구현하고자 하는 것은 비지도 학습의 GCN 모델입니다.

지도학습에서 GCN 모델을 사용했다면, cross entropy를 사용했겠지만, 지금은 비지도 학습이기에 contrastive Learning 등의 기법을 사용해야 할 듯 합니다.

그러나, 현재 파이토치에서 해당 Loss를 제공하고 있지 않아서, 저희가 다시 재정의 해야할 것 같습니다.

In [ ]:
# Loss function 정의

def contrastive_loss(z1, z2, label, margin=1.0):
    """
    z1, z2: 두 노드의 임베딩 벡터
    label: 두 노드가 같은 클래스에 속하는지 여부 (1: 같은 클래스, 0: 다른 클래스)
    margin: 마진 값
    """
    # 유클리드 거리 계산
    distance = F.pairwise_distance(z1, z2)

    # 같은 클래스에 속할 경우 거리의 제곱을 손실로 사용
    loss_same = label * torch.pow(distance, 2)

    # 다른 클래스에 속할 경우 마진까지의 거리의 제곱을 손실로 사용
    loss_diff = (1 - label) * torch.pow(torch.clamp(margin - distance, min=0.0), 2)

    # 손실의 평균을 반환
    loss = torch.mean(loss_same + loss_diff)
    return loss

In [ ]:
# 데이터와 모델 초기화
data = create_data()
model = GCNEncoder(num_features=data.num_features, hidden_dim=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


# 비지도 학습을 위한 모델 훈련
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    embeddings = model(data.x, data.edge_index)
    loss = some_loss_function(embeddings, data)  # 적절한 비지도 손실 함수 정의 필요
    loss.backward()
    optimizer.step()


# 유사성 계산 및 사용자 추천
model.eval()
with torch.no_grad():
    embeddings = model(data.x, data.edge_index)
    # 유사성 계산 로직 추가